TRY
top_k_categorical_accuracy
https://stackoverflow.com/questions/47887533/keras-convolution-along-samples
https://keras.io/layers/wrappers/#timedistributed

In [1]:
input_width = 160
input_height = 100
channels = 3
class_number = 12
data_path = "D:\\Python\\Wormax_learn2\\preprocessed_data_local_notshuffled\\"
model_name = 'models/worm_single_larger_adam_1'
look_forward = 2
import keras

Q:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers, models, optimizers
import keras.backend as K

def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))

def define_model():
    model = models.Sequential()
        
    model.add(layers.Conv2D(32, (3, 3), activation='relu',
                            input_shape=(input_height, input_width, channels)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(class_number, activation='softmax'))    
           
    model.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=[actual_acc])
    
    model.summary()
    return model

In [3]:
model = define_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 158, 32)       896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 79, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 38, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 36, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 18, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 16, 128)        147584    
__________

In [4]:
# little prepocessing
from math import atan2, pi

def get_angle(x, y):
    return atan2(y, x)

def get_direction(x, y, n_classes = 12):
    return round(get_angle(x, y)/2/pi*n_classes)%n_classes

In [5]:
import numpy as np
import os
import random
from functools import reduce
from keras.utils import to_categorical

# Training and validation
data_ratio = 0.7

def generator(data_dir, num_classes, role, shuffle=True, batch_size=128):
    
    listdir = []
    listdir = filter(lambda x: os.path.isfile, os.listdir(data_dir))
    listdir = np.array(list(listdir))
    random.shuffle(listdir)
    
    #print('Found {} files for {}'.format(len(listdir), role))
    
    file_i = 0
    while 1:
        data = np.load(data_dir + listdir[file_i])
        file_i = (file_i+1) if file_i+1<len(listdir) else 0
        
        if role == 'train':
            data = data[:int(round(len(data)*data_ratio))]
        elif role == 'validation':
            data = data[int(round(len(data)*data_ratio)):]
        else:
            raise 'bad role parameter'
        
        # Generating y
        data_targets = np.zeros((len(data)-look_forward, 12))
        for i in range(len(data_targets)):
            data_targets[i] = np.array(to_categorical(get_direction(*data[i+look_forward][1][:2]), num_classes=num_classes))
        data = data[:-1]
        
        # Only X
        data_features = data[:,0]
        
        indexes = np.arange(len(data_features)-1)
        
        if shuffle:
            np.random.shuffle(indexes)
        
        for i in range(0, len(data)-look_forward-batch_size, batch_size):
            samples = data_features.take(indexes[i:i+batch_size],axis=0)
            targets = data_targets.take(indexes[i:i+batch_size],axis=0)
            
            # Weird reshape cuz bug # actually not bug, initial data array is not tensor(idk how to fix)
            samples2 = np.zeros((batch_size, input_height, input_width, channels))
            for j, sample in enumerate(samples):
                samples2[j] = sample
            
            # will not work without this
            samples2 = samples2 / 255
            yield samples2, np.array(targets)

In [6]:
# count class instances count for balancing
if False:
    i = 0
    classes = np.zeros((class_number))
    for samples, targets in generator(data_path, class_number, 'train', batch_size=1024):
        for j in targets:
            classes += j
        i += 1
        if i == 1000:
            break
    classes

In [ ]:
import numpy as np

train_generator = generator(data_path, class_number, 'train', batch_size=16)
validation_generator = generator(data_path, class_number, 'validation', batch_size=32)

print(next(train_generator)[0].shape)
print(next(train_generator)[1].shape)

(16, 100, 160, 3)
(16, 12)


### tensorboard --logdir=D:\Python\Keras\Wormax\log_dir

In [ ]:
class_weight = {0: 1.0,
                 1: 1.62,
                 2: 2.68,
                 3: 3.36,
                 4: 2.51,
                 5: 1.53,
                 6: 1.0,
                 7: 1.37,
                 8: 2.16,
                 9: 2.61,
                 10: 2.04,
                 11: 1.3}

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_dir\\' + model_name
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name + '.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.1,                              
        patience=50, 
        min_lr=0.00001
    )
]

steps_per_epoch = 100
history = model.fit_generator(train_generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=500,
                            validation_data=validation_generator,
                            validation_steps=int(round(steps_per_epoch/data_ratio*(1-data_ratio))),
                            shuffle=True,
                            class_weight=class_weight,
                            callbacks=callbacks
                        )

Epoch 1/500
100/100 [==============================] - 11s 109ms/step - loss: 3.5749 - actual_acc: 0.2450 - val_loss: 2.5968 - val_actual_acc: 0.0727
Epoch 2/500
100/100 [==============================] - 12s 122ms/step - loss: 2.7879 - actual_acc: 0.4363 - val_loss: 2.5831 - val_actual_acc: 0.1214
Epoch 3/500
100/100 [==============================] - 4s 41ms/step - loss: 2.2739 - actual_acc: 0.5356 - val_loss: 2.3894 - val_actual_acc: 0.2115
Epoch 4/500
100/100 [==============================] - 13s 128ms/step - loss: 2.1345 - actual_acc: 0.5825 - val_loss: 3.1490 - val_actual_acc: 0.2398
Epoch 5/500
100/100 [==============================] - 12s 124ms/step - loss: 3.1859 - actual_acc: 0.3575 - val_loss: 2.1989 - val_actual_acc: 0.2173
Epoch 6/500
100/100 [==============================] - 13s 126ms/step - loss: 3.2978 - actual_acc: 0.3631 - val_loss: 2.2515 - val_actual_acc: 0.2420
Epoch 7/500
100/100 [==============================] - 4s 40ms/step - loss: 2.7408 - actual_acc: 0.462

100/100 [==============================] - 4s 40ms/step - loss: 2.6712 - actual_acc: 0.4856 - val_loss: 1.9761 - val_actual_acc: 0.2798
Epoch 110/500
100/100 [==============================] - 14s 144ms/step - loss: 2.2097 - actual_acc: 0.5588 - val_loss: 1.6214 - val_actual_acc: 0.4564
Epoch 111/500
100/100 [==============================] - 12s 122ms/step - loss: 2.1760 - actual_acc: 0.5787 - val_loss: 1.6161 - val_actual_acc: 0.4426
Epoch 112/500
100/100 [==============================] - 4s 41ms/step - loss: 1.6948 - actual_acc: 0.6631 - val_loss: 1.9332 - val_actual_acc: 0.3699
Epoch 113/500
100/100 [==============================] - 24s 235ms/step - loss: 2.9060 - actual_acc: 0.4131 - val_loss: 1.9717 - val_actual_acc: 0.3430
Epoch 114/500
100/100 [==============================] - 13s 126ms/step - loss: 3.0471 - actual_acc: 0.3769 - val_loss: 1.9465 - val_actual_acc: 0.2994
Epoch 115/500
100/100 [==============================] - 16s 157ms/step - loss: 2.8372 - actual_acc: 0.426

In [ ]:
from keras import backend as K
K.clear_session()